# Martech - Load Data

## Initialise Connection

In [10]:
from astra_db import AstraSession

astra = AstraSession()

In [11]:
# Prepare Statements
insert_communication_stmt = astra.session.prepare("INSERT INTO communications (communication_id, comm_date, customer_id, channel, category_group, category, activity_name) VALUES (?, ?, ?, ?, ?, ?, ?)")
update_acty_cap_stmt = astra.session.prepare("INSERT INTO comm_by_activity_day (activity_name, comm_date_bucket, comm_time_bucket) VALUES (?, ?, ?)")

## Load Data

### Define Entities

In [12]:
# Define Entities

# customer_id
customer_id_array = [
    '0b6ec30f-8b1a-406f-82f6-89222a017449',
    '8435a9b9-9e88-495c-a297-dfd25d44428b',
    '8e12d570-e47d-43b9-9435-b2020899860b',
    '9e53cd14-0316-46ff-b913-30228fcc5b73',
    'a6e198e0-674d-4a27-ba01-595a78fda2ef',
    'af9e65bd-4474-40c9-8d16-7b82035d3e0f',
    'b191cef3-4624-4e88-9a93-73de506e4be7',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
    'cf3a5d68-e66f-4ee7-a2d1-2666c5366540',
    'f68e8bc9-86b7-4c8e-ba74-87d31d27a531',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
]

# channel
channel_array = ['email', 'sms', 'push', 'in-app', 'direct']

# category_group
category_group_array = ['ctgy_grp_1', 'ctgy_grp_2', 'ctgy_grp_3', 'ctgy_grp_4', 'ctgy_grp_5']

# category
category_array = ['ctgy_1', 'ctgy_2', 'ctgy_3', 'ctgy_4', 'ctgy_5']

# activity_name
activity_name_array = ['credit_card', 'mortgage', 'business', 'retail', 'investment']

### Define Insert Function

In [13]:
from datetime import datetime
import random
import uuid

YEAR = 2025
MONTH = 2


def insert_comm():
    #sec = random.randint(0, 59)
    #ms = random.randint(0, 999999)
    #date = datetime(YEAR, MONTH, day, hour, min, sec, ms)
    date = datetime.now()
    comm_date_bucket = int(date.strftime('%Y%m%d')) # bucket by day
    comm_time_bucket = int(date.strftime('%H%M%S%f')[:8]) # format time as HHMMSSssss
    #print(f"{comm_date_bucket} - {comm_time_bucket}")
    
    
    # generate values
    comm_id = str(uuid.uuid4())
    customer_id = customer_id_array[random.randint(0, 9)]
    channel = channel_array[random.randint(0, 4)]
    category_group = category_group_array[random.randint(0, 4)]
    category = category_array[random.randint(0, 4)]
    activity_name = activity_name_array[random.randint(0, 4)]


    # insert communication
    astra.session.execute(
        insert_communication_stmt, 
        (comm_id, date, customer_id, channel, category_group, category, activity_name)
    )
    
    astra.session.execute(
        update_acty_cap_stmt, 
        (activity_name, comm_date_bucket, comm_time_bucket)
    )

    

    """
    # Update Profile Cap
    astra.session.execute(
        astra.update_profile_cap_stmt, 
        (customer_id, channel, comm_date_bucket, comm_id)
    )

    # Update Acty Cap
    astra.session.execute(
        astra.update_acty_cap_stmt, 
        (activity_name, comm_date_bucket, comm_id)
    )

    # Update Prty Cap
    astra.session.execute(
        astra.update_prty_cap_stmt, 
        (category_group, category, comm_date_bucket, comm_id)
    )

    # Update Channel Cap
    astra.session.execute(
        astra.update_channel_cap_stmt, 
        (channel, comm_date_bucket, comm_id)
    )
    """

### Load Single

In [ ]:
insert_comm()

### Load Many

In [17]:
for i in range(10):
    insert_comm()

### Load a month

In [ ]:
# iterate over month
for day in range(1, 31):
    print()
    print(day)

    # iterate over hours in a day
    for hour in range(0, 24):
        print(f"{hour} ", end='')
        insert_comm()

## Clean Up

In [18]:
astra.shutdown()